In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os.path
import os
from scipy.interpolate import interp1d
from scipy.signal import butter,filtfilt
from scipy.optimize import curve_fit
from scipy import odr

#%matplotlib inline 
%matplotlib qt
%run Functions_for_RUS_LabView.ipynb

# Import data

Select folder/ filepath where data files are located

In [2]:
folder = "C:\\Users\\j111\\Box Sync\\Projects"
folder = "C:\\Users\\Florian\\Box Sync\\Projects"
project = "\\Mn3Ge\\RUS\\Mn3Ge_2001B\\temp_dependent_data_with_Labview\\improved_setup\\good_data"
# project = "\\Mn3Ge\\RUS\\Mn3Ge_2001B\\temp_dependent_data_with_Labview\\improved_setup\\new_data"
# project = "\\Mn3.019Sn0.981\\RUS\\2007A\\good_data"
project = "/UTe2/RUS/UTe2_2007C/PLL/good data"
# project = "/UTe2/RUS/UTe2_2007C/Red Pitaya/good_data"


data_files = os.listdir(folder+project)
data_files = [i for i in data_files if i[-4:]=='.dat']
np.array(data_files)

array(['1035kHz_down.dat', '1150kHz_down_clean.dat',
       '1221kHz_up_clean.dat', '1275kHz_up_clean.dat', '1355kHz_down.dat',
       '1428kHz_down_clean.dat', '1508kHz_down_clean.dat',
       '1677kHz_up_clean.dat', '1702kHz_up_clean.dat',
       '1805kHz_up_clean.dat', '1894kHz_up_clean.dat',
       '1968kHz_down_clean.dat', '2003kHz_up_clean.dat',
       '2096kHz_up_clean.dat', '2176kHz_down.dat', '2234kHz_up_clean.dat',
       '2322kHz_down_clean.dat', '2338kHz_up.dat',
       '2367kHz_down_clean.dat', '2420kHz_down.dat',
       '2454kHz_up_clean.dat', '2475kHz_up_clean.dat', '2502kHz_up.dat',
       '2522kHz_up_clean.dat', '2578kHz_down_clean.dat', '2585kHz_up.dat',
       '2641kHz_up_clean.dat', '2663kHz_up_clean.dat', '2706kHz_down.dat',
       '2750kHz_down.dat', '2791kHz_down.dat', '2824kHz_up_clean.dat',
       '2865kHz_up.dat', '2919kHz_down.dat', '3091kHz_up_clean.dat',
       '3107kHz_up.dat', '3397kHz_up_clean.dat', '909kHz_up_clean.dat'],
      dtype='<U22')

## actually import data
(you usually don't have to change anything here)

In [3]:
filenames = [folder + project + '\\' + i for i in data_files[:]]
Traw = []
fraw = []
gammaraw = []

for name in filenames:
    individual_data = []
    T, f= import_data(name, 1, 2)    
    Traw.append(np.array(T))
    fraw.append(np.array(f))
    #gammaraw.append(np.array(gamma))
    
# write data into seperate arrays for temperature "T" and frequency "f"
Traw = np.array(Traw) # temperature in K
fraw = np.array(fraw) # frequencies in kHz
#gammaraw = np.array(gammaraw) # resonance widths in kHz

In [4]:
# #folder = "C:\\Users\\j111\\Box Sync\\Projects"
# folder = "C:\\Users\\Florian\\Box Sync\\Projects"
# project = "\\Mn3Ge\\RUS\\sample_with_green_face_from_010920\\raw_data.txt"
# text = 'None'
# data = []
# for i in np.arange(len(Traw)):
#     data.append(Traw[i])
#     data.append(fraw[i])
#     data.append(gammaraw[i])
# save(data, text, folder+project)

In [5]:
Tmin = 1.25
Tmax = 1.9
# Tmin = 407
# Tmax = 425
# Tmin = 355
# Tmax = 395
mask = np.array([(i > Tmin) & (i < Tmax) for i in Traw])
labels = ([str(np.round(i[-1], 0))[:-2]+' kHz' for i in fraw])


shift = [0, -0.1, -.7, 0, 0, 0, 0, 0]
shift = [8, 8, 6, 5, 4, 3, 2, 3, -3.5]

plt.figure(figsize=(9.5,7))

palette = plt.get_cmap('viridis')
for i in np.arange(len(Traw)):
    # plt.scatter(Traw[i][mask[i]], fraw[i][mask[i]]*1e3, s=70, color=palette(i/(len(Traw)-1)), label=labels[i])
    plt.plot(Traw[i][mask[i]], fraw[i][mask[i]], color=palette(i/(len(Traw)-1)), label=labels[i], linewidth=5)
    
plt.xlabel('T (K)',fontsize=22)
plt.ylabel('f (Hz)',fontsize=22)
plt.xlim(Tmin-.01, Tmax+.01)
#plt.legend(fontsize=18, markerscale=8)
plt.tick_params(axis="both",direction="in", labelsize=20, bottom='True', top='True', left='True', right='True', 
                length=4, width=1, which = 'major')

plt.show()

In [6]:
Tmin = 1.29
Tmax = 1.65
# Tmin = 407
# Tmax = 422.5
# Tmin = 355
# Tmax = 395
mask = np.array([(i > Tmin) & (i < Tmax) for i in Traw])
# labels = ([str(np.round(i[-1]/1e3, 0))[:-2]+' kHz' for i in fraw])
labels = ([str(np.round(i[-1], 0))[:-2]+' kHz' for i in fraw])
# labels = ['most A1g1', 'most A1g2', 'most A1g3', 'most E1g', 'most E2g']


shift = [0, -0.1, -.7, 0, 0, 0, 0, 0]
shift = [8, 8, 6, 4.8, 2.8, 2.5, 2.3, 2.3, -3.5]
shift = [5, 4.6, 3.5, 1.7, 1.7, 0.5, 0, -2.1]
shift = [0, -6, 0, 0, 0, 0, 0]
# shift = [0, 3.5, 0, 0, 0, 1.7, 1.3, 0, -0.2, 0, 0]

plt.figure(figsize=(9.5,7))
palette = plt.get_cmap('viridis')
k=0
for i in np.arange(len(Traw)):
# for i in [4, 12, 16, 1, 5]:

# for i in [1,5,6,8]:
    # plt.scatter(Traw[i][mask[i]], ((fraw[i]-fraw[i][-1])/fraw[i][-1]*1e5)[mask[i]], s=10, color=palette(i/(len(Traw)-1)), label=labels[i])
    plt.scatter(Traw[i][mask[i]], ((fraw[i]-fraw[i][-1])*1e-3)[mask[i]], s=10, color=palette(i/(len(Traw)-1)), label=labels[i])

    # plt.plot(Traw[i][mask[i]], ((fraw[i]-fraw[i][-1])/fraw[i][-1]*1e5)[mask[i]], linewidth=5, color=palette(i/2/(len(Traw)/2-1)), label=labels[i])
    # plt.scatter(Traw[i][mask[i]], ((fraw[i]-fraw[i][-1])/fraw[i][-1]*1e2)[mask[i]], s=5, label=labels[k])
    
    # plt.scatter(Traw[i][mask[i]], ((fraw[i]-fraw[i][-1])/fraw[i][-1]*1e5)[mask[i]]+shift[i], s=10, color=palette((k)/(3)), label=labels[i])
    k=k+1

# for i in [1, 2, 4, 5]:
#     plt.scatter(Traw[i][mask[i]], ((fraw[i]-fraw[i][-1])/fraw[i][-1]*1e5)[mask[i]]+shift[i], s=10, color='gray')

plt.xlabel('T (K)',fontsize=22)
# plt.ylabel('$\\mathrm{\\Delta f / f}$ $(10^{-5})$',fontsize=22)
plt.ylabel('$\\mathrm{\\Delta f / f}$ $(10^{-5})$',fontsize=22)
plt.xlim(Tmin-0.01, Tmax+0.01)
#plt.legend(fontsize=18, markerscale=4)
plt.tick_params(axis="both",direction="in", labelsize=20, bottom='True', top='True', left='True', right='True', 
                length=4, width=1, which = 'major')

# plt.ylim(-1,6)
# plt.legend(fontsize=20)

plt.show()

# Interpolate the data

In [94]:
# this creates interpolated frequency data at the same temperature points in the biggest temperature
# range possible (all frequency curves have to be measured in at least this temperature range)
Tmin_int = max([min(t) for t in Traw])
Tmax_int = min([max(t) for t in Traw])
n = np.mean(np.array([len(t) for t in Traw]))
Tint = np.linspace(Tmin_int, Tmax_int, int(n))

fint = []
gint = []
# for i in np.arange(len(Traw)):
#     fi, gi = interpolate(Traw[i], fraw[i], gammaraw[i], Tint, 'linear')
#     fint.append(fi)
#     gint.append(gi)

for i in np.arange(len(Traw)):
    fint.append( interp1d(Traw[i], fraw[i], kind='linear')(Tint) )

fint = np.array(fint)
dfint = np.array([(i-i[-1]) for i in fint])
dfofint = np.array([(i-i[-1]/i) for i in fint])
#gint = np.array(gint)


# mask the raw data arrays such that df is zero at the same temperature
mask_raw = np.array([(i >= Tmin_int) & (i <= Tmax_int) for i in Traw])
Traw = np.array([Traw[i][mask_raw[i]] for i in np.arange(len(Traw))])
fraw = np.array([fraw[i][mask_raw[i]] for i in np.arange(len(Traw))])
dfraw = np.array([(i-i[-1]) for i in fraw])
dfofraw = np.array([(i-i[-1])/i[-1] for i in fraw])
#graw = np.array([gammaraw[i][mask_raw[i]] for i in np.arange(len(Traw))])

print('the maximum temperature range is '+str(Tmin_int)+' to '+str(Tmax_int)+' K')

the maximum temperature range is 356.9127 to 394.7912 K


# Apply a low-pass filter to the interpolated data

In [95]:
Tlp = Tint
flp = np.array([butter_lowpass_filter_ready(f, 2.5) for f in fint])
dflp = np.array([(i-i[-1]) for i in flp])
dfoflp = np.array([(i-i[-1]/i) for i in flp])

In [89]:
Tlp = Tint
flp = fint
dflp = dfint
dfoflp = dfofint

# Plot the resulting curves over the raw data to see if it worked

### plot the frequency vs T (plots filtered data and raw data)

In [110]:
# this just plots the filtered frequency over the raw data

Tmin = 410
Tmax = 420
Tmin = 355
Tmax = 390

# which frequencies do you want to plot
resonances = np.arange(len(flp))
#resonances = [0,5,10,12]


plt.figure(figsize=(10,7))

# make scatter plot of raw data
mask_raw = np.array([(i >= Tmin) & (i <= Tmax) for i in Traw])

#shift = [.3, 0, .4, -0.3, -1]
#index = [1, 3, 5, 9, 13]
#shift = 7
for i in resonances:
    palette = plt.get_cmap('brg')
    plt.scatter(Traw[i][mask_raw[i]], dfraw[i][mask_raw[i]], color=palette(i/(len(flp)-1)), s=10)
    

# make plot of interpolation
mask_lp = (Tint >= Tmin) & (Tint <= Tmax)

shift = 7
for i in resonances:
    palette = plt.get_cmap('Set1')
    plt.plot(Tlp[mask_lp], dflp[i][mask_lp], color=palette(i/len(flp)), label=(str(round(fraw[i][-1], 1))+', '+str(i)))


plt.xlabel('T (K)',fontsize=20)
plt.ylabel('$\mathrm{\Delta f\,\,(kHz)}$',fontsize=20)
plt.tick_params(axis="both",direction="in", labelsize=18, bottom='True', top='True', left='True', right='True', 
                length=4, width=1, which = 'major')

#plt.ylim([-40, 20])
#plt.xlim(360, 380)

plt.legend(loc=(.6, .04), fontsize=17)

### plot the width vs T (plots intepolated data and raw data)

In [111]:
# # this just plots the interpolated width over the raw data
# Tmin = 350
# Tmax = 450

# # which frequencies do you want to plot
# resonances = np.arange(len(fint))
# #resonances = [0,5,10,12]

# plt.figure(figsize=(10,10))

# # make scatter plot of raw data
# mask_raw = np.array([(i >= Tmin) & (i <= Tmax) for i in Traw])

# shift = 1
# for i in resonances:
#     palette = plt.get_cmap('brg')
#     plt.scatter(Traw[i][mask_raw[i]], graw[i][mask_raw[i]]+shift, s=10, color=palette(i/(len(Traw)-1)))
#     shift=shift+1

    
# # make plot of interpolation
# mask_int = (Tint >= Tmin) & (Tint <= Tmax)

# shift = 1
# for i in resonances:
#     palette = plt.get_cmap('Set1')
#     plt.plot(Tint[mask_int], gint[i][mask_int]+shift, c=palette(i/(len(gint)-1)), label=(str(round(fint[i][-1], 1))+', '+str(i)) )
#     shift=shift+1

# plt.xlabel('T (K)',fontsize=15)
# plt.ylabel('$\mathrm{\Gamma}$ (kHz) ',fontsize=15)
# plt.tick_params(axis="both",direction="in", labelsize=15, bottom='True', top='True', left='True', right='True', 
#                 length=4, width=1, which = 'major')

# #plt.ylim(1.5,3)

# plt.legend(loc=(.7, .4), fontsize=15)

# Import rpr_out file

In [278]:
folder = "C:\\Users\\j111\\Box Sync\\Projects"
folder = "C:\\Users\\Florian\\Box Sync\\Projects"
project = "\\Mn3Ge\\RUS\\Mn3Ge_2001B\\high_T_200113\\elastic_constants_gradient_fit\\5_um_error\\mean.dat"
# project = "\\Mn3.019Sn0.981\\RUS\\2007A\\Mn3.019Sn0.981_2007A_436K_out.dat"
# project = "\\Mn3.019Sn0.981\\RUS\\2007A\\Mn3.019Sn0.981_2007A_441K_out.dat"
# project = "\\Mn3.019Sn0.981\\RUS\\2007A\\Mn3.019Sn0.981_2007A_438K_out.dat"

name = folder + project
data = []

# here you import the a matrix elements allowing for conversion of deltaf/f to deltac/c
f = open(name, 'r')
for i in np.arange(6):
    f.readline()

for line in f:
    line=line.strip()
    line=line.split()
    
    relevant_data = []
    if len(line) == 12:
        if len(line[0]) == 1 or len(line[0]) == 2:
            for i in [1, 2, 7, 8, 9, 10, 11]:
                relevant_data.append(float(line[i]))
            
            data.append(np.array(relevant_data))
    
data = np.array(data)

fht_exp = data[:, 0]
fht_calc = data[:, 1]
# logarithmic derivative dlnf/dlncij
# order is c33, c23, c12, c44, c66
df_dc = data[:, 2:7]

# high temperature elastic constants from RPR fit in GPa
# order is the same as for logarithmic derivatives
# Cht = [33, 23, 12, 44, 66]
data = []

f = open(name, 'r')
for i in np.arange(6):
    f.readline()

for line in f:
    line=line.strip()
    line=line.split()
    
    if len(line)>0 and np.array(line)[0][0]=='c':
        data.append(line)
        
data = np.array(data)

Cht = np.array( [ float(data[1][1]), float(data[2][3]), float(data[0][1])-2*float(data[5][1]), float(data[4][1]), float(data[5][1]) ] )

## Now you have to decide which f(T) curve belongs to which resonance at high T

In [279]:
# this gives you the high T frequencies of the f(T) curves
label = np.array([round(f[-1],1) for f in flp])
print (label)

[1320.4 1362.  1385.4 1454.1 1627.5 1652.9 1668.6 1709.9 1821.  1857.5
 1879.4 1908.5 1954.4 2051.1 2725.9 2825.2 2856.8 2871.7]


In [280]:
print([[fht_exp[i], i] for i in np.arange(len(fht_exp))][:50])

[[0.8632903, 0], [1.3159182, 1], [1.3377351, 2], [1.3597448, 3], [1.380895, 4], [1.4513183, 5], [1.6276308, 6], [1.6520975, 7], [1.6675098, 8], [1.706839, 9], [1.8195792, 10], [1.8583015, 11], [1.876659, 12], [1.9052377, 13], [1.9147265, 14], [1.9502579, 15], [2.043762, 16], [2.056432, 17], [2.0737541, 18], [2.1540241, 19], [2.2591281, 20], [2.2678907, 21], [2.2821, 22], [2.2867196, 23], [2.3316123, 24], [2.3534091, 25], [2.397702, 26], [2.4052324, 27], [2.452348, 28], [2.4810884, 29], [2.4957685, 30], [2.5109429, 31], [2.5730035, 32], [2.5871172, 33], [2.6456351, 34], [2.6608291, 35], [2.707036, 36], [2.7189031, 37], [2.7871687, 38], [2.8190637, 39], [2.8537569, 40], [2.8677502, 41], [2.9004674, 42], [3.0049644, 43], [3.027684, 44], [3.0554662, 45], [3.0753968, 46], [3.12169, 47], [3.1539707, 48], [3.1804593, 49]]


In [281]:
#n = [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 18]
n = [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 36, 40, 41, 42] #Mn3Ge
# n = [1, 2, 4, 5, 7, 8, 9, 11, 14, 15, 20, 22, 29, 30, 33, 34] #Mn3.019Sn0.981 2007 A

fht = fht_exp[n]
#fht = fht_calc[n] # this is not what I usually want!!!!!!!!!! usually it is fht_exp
a = df_dc[n]
#ffinal = np.array([ ffinal[i-1] for i in n ])

In [282]:
np.array([flp[i, -1] for i in np.arange(len(flp))])

array([1320.40394242, 1362.02924698, 1385.36246377, 1454.12761501,
       1627.49866564, 1652.85783815, 1668.59123894, 1709.8556143 ,
       1820.97630903, 1857.47756175, 1879.36471396, 1908.4724541 ,
       1954.42215036, 2051.13112946, 2725.91244047, 2825.23007199,
       2856.78661253, 2871.66906797])

In [283]:
fht

array([1.3159182, 1.3597448, 1.380895 , 1.4513183, 1.6276308, 1.6520975,
       1.6675098, 1.706839 , 1.8195792, 1.8583015, 1.876659 , 1.9052377,
       1.9502579, 2.043762 , 2.707036 , 2.8537569, 2.8677502, 2.9004674])

# this is where C(T) is calculated

In [284]:
CofT = elastic_constants (flp/1000, fht, a, Cht)

In [285]:
# order in Cht and thus in CofT is [33, 23, 12, 44, 66]
# order in Cirrep is A1g1, A1g2, A1g3, E1g, E2g

label_irrep = ['A1g,1', 'A1g,2', 'A1g,3', 'E1g', 'E2g'] # this is for the right legend in subsequent plots

Cirrep = np.array([ (CofT[4]+CofT[2]), CofT[0], CofT[1], 4*CofT[3], CofT[4] ])
dCirrepoC = np.array([(C-C[-1])/C[-1] for C in Cirrep ])
dCirrep = np.array([(C-C[-1]) for C in Cirrep ])

In [286]:
Cht

array([194.58691,  14.29768,  42.47116,  45.20971,  48.16944])

In [287]:
folder = "C:\\Users\\j111\\Box Sync\\Projects"
folder = "C:\\Users\\Florian\\Box Sync\\Projects"
project = "\\Mn3Ge\\RUS\\Mn3Ge_2001B\\high_T_200113\\elastic_constants_gradient_fit\\5_um_error\\irreducible_elastic_constants_err6.txt"
# project = "\\Mn3.019Sn0.981\\RUS\\2007A\\irreducible_elastic_constants.txt"
text = [['T', 'A1g1', 'A1g2', 'A1g3', 'E1g', 'E2g'], ['K', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa']]
data = np.transpose( np.vstack( [Tlp, Cirrep] ) ) 
# save(data, text, folder+project)

In [288]:
Tmin = 400
Tmax = 450
Tmin = 350
Tmax = 395

mask = np.array([(i >= Tmin) & (i <= Tmax) for i in Tlp])


# plt.figure(figsize=(10,6))

# for i in np.arange(len(Cirrep)):
#     plt.plot(Tlp[mask], Cirrep[i][mask], linewidth=2.5, label=label_irrep[i])
    
# plt.legend()
# plt.xlabel('T (K)',fontsize=15)
# plt.ylabel('$\mathrm{C_{irrep}}$ (GPa) ',fontsize=15)
# plt.tick_params(axis="both",direction="in", labelsize=15, bottom='True', top='True', left='True', right='True', 
#                 length=4, width=1, which = 'major')

In [289]:
index1 = [0,1,2]
index2 = [3,4]


f1 = plt.figure(figsize=(10,6))
shift = [-.2,0.5,0,0,-4]
for i in np.arange(len(Cirrep)):
#for i in index1:
    plt.plot(Tlp[mask], dCirrep[i][mask]+shift[i], linewidth=2.5, label=label_irrep[i])
    
#plt.ylim([-50,20])
plt.legend(loc=(.1, 0.1), fontsize=15)
plt.xlabel('T (K)',fontsize=15)
plt.ylabel('$ \mathrm{\Delta C_{irrep}}$ (GPa)',fontsize=15)
plt.tick_params(axis="both",direction="in", labelsize=15, bottom='True', top='True', left='True', right='True', 
                length=4, width=1, which = 'major')

f2 = plt.figure(figsize=(10,6))
#for i in np.arange(len(Cirrep)):
for i in index2:
    plt.plot(Tlp[mask], dCirrep[i][mask], linewidth=2.5, label=label_irrep[i])
    
#plt.ylim([-50,20])
plt.legend(loc=(.1, 0.1), fontsize=15)
plt.xlabel('T (K)',fontsize=15)
plt.ylabel('$ \mathrm{\Delta C_{irrep}}$ (GPa)',fontsize=15)
plt.tick_params(axis="both",direction="in", labelsize=15, bottom='True', top='True', left='True', right='True', 
                length=4, width=1, which = 'major')

In [299]:
Cht_irrep = np.array( [ Cht[2]+Cht[4], Cht[0], Cht[1], 4*Cht[3], Cht[4] ] )
at = np.transpose(a)
a_irrep = np.transpose( np.array( [ at[2]*Cht_irrep[0]/Cht[2], at[0]*Cht_irrep[1]/Cht[0], at[1]*Cht_irrep[2]/Cht[1], at[3]/4*Cht_irrep[3]/Cht[3], (at[4]/Cht[4]-at[2]/Cht[2])*Cht_irrep[4] ]) )

In [328]:
i = 0
value = 0
mode = 4
for k in np.arange(len(a_irrep)):
    if a_irrep[k,mode]>value:
        value = a_irrep[k,mode]
        i = k

print (i, a_irrep[i])

5 [ 0.03032653  0.00405    -0.00211     0.00379     0.96394347]


# Save all data

In [351]:
folder = "C:\\Users\\j111\\Box Sync\\Projects"
project = "\\Mn3Ge\\RUS\\Mn3Ge_2001B\\high_T_200113\\elastic_constants_gradient_fit\\irreducible_elastic_constants_mean.txt"
# project = "\\Mn3.019Sn0.981\\RUS\\2007A\\irreducible_elastic_constants.txt"
text = [['T', 'A1g1', 'A1g2', 'A1g3', 'E1g', 'E2g'], ['K', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa']]
data = np.transpose( np.vstack( [Tlp, Cirrep] ) ) 
save(data, text, folder+project)

In [188]:
folder = "C:\\Users\\j111\\Box Sync\\Projects"
#folder = "C:\\Users\\Florian\\Box Sync\\Projects"
project = "\\Mn3Ge\\RUS\\sample_with_green_face_from_010920\\change_in_irreducible_elastic_constants.txt"
text = [['T', 'A1g1', 'A1g2', 'A1g3', 'E1g', 'E2g'], ['K', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa']]
data = np.transpose( np.vstack( [Tlp, dCirrep] ) ) 
#save(data, text, folder+project)

In [189]:
folder = "C:\\Users\\j111\\Box Sync\\Projects"
#folder = "C:\\Users\\Florian\\Box Sync\\Projects"
project = "\\Mn3Ge\\RUS\\sample_with_green_face_from_010920\\resonance_frequencies.txt"
text = [['T']+['f' for i in flp], ['K']+['kHz' for i in flp]]
data = np.transpose( np.vstack( [Tlp, flp] ) ) 
#save(data, text, folder+project)

In [271]:
folder = "C:\\Users\\j111\\Box Sync\\Projects"
#folder = "C:\\Users\\Florian\\Box Sync\\Projects"
project = "\\Mn3Ge\\RUS\\sample_with_green_face_from_010920\\resonance_widths.txt"
text = [['T']+['Gamma' for i in gint], ['K']+['kHz' for i in gint]]
data = np.transpose( np.vstack( [Tint, gint] ) ) 
#save(data, text, folder+project)

In [ ]:
# calculate a for irreducible reps
# order in Cht and thus in CofT is [33, 23, 12, 44, 66]
# order in Cirrep is A1g1, A1g2, A1g3, E1g, E2g
Ciht = np.array([Cht[2]+Cht[4], Cht[0], Cht[1], 4*Cht[3], Cht[4]])
airr = np.array([ [ b[2]/Cht[2]*Ciht[0], b[0]/Cht[0]*Ciht[1], b[1]/Cht[1]*Ciht[2], 0.25*b[3]/Cht[3]*Ciht[3], 
                   Ciht[4]*(b[4]/Cht[4] - b[2]/Cht[2]) ] for b in a])

folder = "C:\\Users\\j111\\Box Sync\\Projects"
#folder = "C:\\Users\\Florian\\Box Sync\\Projects"
project = "\\Mn3Ge\\RUS\\sample_with_green_face_from_010920\\logarithmic_derivative_a_irreducible.txt"
text = [['f (kHz)', 'aA1g1', 'aA1g2', 'aA1g3', 'aE1g', 'aE2g']]
data = np.array([np.append(fht[i], airr[i]) for i in np.arange(len(fht))])
#save(data, text, folder+project)

In [192]:
folder = "C:\\Users\\j111\\Box Sync\\Projects"
folder = "C:\\Users\\Florian\\Box Sync\\Projects"
project = "\\Mn3Ge\\RUS\\sample_with_green_face_from_010920\\elastic_constants_NOT_irreps.txt"
project = "\\Mn3.019Sn0.981\\RUS\\2007A\\elastic_constants_NOT_irreps.txt"
text = [['T', '33', '23', '12', '44', '66'], ['K', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa']]
data = np.transpose( np.vstack( [Tlp, CofT] ) )
#save(data, text, folder+project)

# estimate the error for irreducible elastic constants

In [193]:
Tmin = 368
Tmax = 372

mask = np.array([(i >= Tmin) & (i <= Tmax) for i in Tfinal])

# Filter requirements.
period = 5.0         # Sample Period
fs = 50       # sample rate, Hz
cutoff = .7   # desired cutoff frequency of the filter, Hz , slightly higher than actual 1.2 Hz

nyq = 0.5 * fs  # Nyquist Frequency

order = 2       # sin wave can be approx represented as quadratic
n = int(period * fs) # total number of samples


def butter_lowpass_filter(data, cutoff, fs, order):
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y



Tlp = Tfinal[mask]
Clp = butter_lowpass_filter(dCirrep[4][mask], cutoff, fs, order)

plt.figure(figsize=(9,6))

plt.scatter(Tfinal[mask], dCirrep[4][mask])
plt.plot(Tlp, Clp, c='orange')

rms = np.sqrt( sum( abs(dCirrep[4][mask]-Clp)**2 ) / len(Clp) )
print (rms)

NameError: name 'Tfinal' is not defined